In [17]:
import numpy as np
import pandas as pd
import requests
import folium

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(data, 'html5')

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [14]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
df.shape

(103, 3)

In [7]:
df.to_csv('toroto.csv')

In [8]:
# I HAVE ALREADY PREPROCESSING THE DATA IN EXCEL

In [29]:
data = pd.read_csv('data.csv')

In [30]:
data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Postal Code   103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [36]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


In [39]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [40]:
CLIENT_ID = '4WUKVVO3SKQZSR4YVWR2VKIKO0YCGLR0NLJ1HEOLFFQTDVWK' # your Foursquare ID
CLIENT_SECRET = '5HW01QZRZVG3KCFNGCXKM2EC3SZSCFIQURUMRZIS4SQDMITG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4WUKVVO3SKQZSR4YVWR2VKIKO0YCGLR0NLJ1HEOLFFQTDVWK
CLIENT_SECRET:5HW01QZRZVG3KCFNGCXKM2EC3SZSCFIQURUMRZIS4SQDMITG


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
venues_tor = getNearbyVenues(data['Neighborhood'], data['Latitude'], data['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


In [45]:
venues_tor.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,Bank


In [46]:
venues_cat = pd.get_dummies(venues_tor['Venue Category'])
venues_cat.head()

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
venues_cat['Neighbourhood'] = venues_tor['Neighbourhood']
venues_cat.head()

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Malvern, Rouge"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Malvern, Rouge"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Rouge Hill, Port Union, Highland Creek"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Rouge Hill, Port Union, Highland Creek"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Guildwood, Morningside, West Hill"


In [52]:
grouped_df = venues_cat.groupby('Neighbourhood').mean().reset_index()
grouped_df.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighbourhood'] = grouped_df['Neighbourhood']

for ind in np.arange(grouped_df.shape[0]):
    venues_sorted.iloc[ind, 1:] = most_common_venues(grouped_df.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Clothing Store,Skating Rink,Accessories Store,Monument / Landmark,Medical Center,Mediterranean Restaurant,Men's Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pub,Pool,Playground,Sandwich Place,Medical Center,Mediterranean Restaurant,Modern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Bridal Shop,Supermarket,Restaurant,Fried Chicken Joint,Mobile Phone Shop,Diner,Sandwich Place,Middle Eastern Restaurant
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Accessories Store,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Comfort Food Restaurant,Pharmacy,Pub,Restaurant,Café,Butcher


In [74]:
from sklearn.cluster import KMeans

In [75]:
k_model = KMeans(n_clusters=5).fit(grouped_df.drop('Neighbourhood',1))
k_model.labels_[:10]

array([2, 0, 0, 0, 0, 2, 0, 0, 2, 1])

In [92]:
venues_sorted.insert(0, 'Cluster Labels', k_model.labels_)

tor_data = data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tor_data = tor_data.join(venues_sorted.set_index('Neighbourhood'), on='Neighborhood')
tor_data.dropna(inplace=True)

tor_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Print Shop,Fast Food Restaurant,Accessories Store,Modern European Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Home Service,Accessories Store,Monument / Landmark,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Medical Center,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Mexican Restaurant,Restaurant,Donut Shop,Bank,Mobile Phone Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Indian Restaurant,Accessories Store,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Caribbean Restaurant,Bakery,Bank,Gas Station,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Lounge,Hakka Restaurant,Men's Store


In [104]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Neighborhood'], tor_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [94]:
tor_data.loc[tor_data['Cluster Labels'] == 0, tor_data.columns[[1] + list(range(5, tor_data.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0.0,Coffee Shop,Korean BBQ Restaurant,Indian Restaurant,Accessories Store,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
6,Scarborough,0.0,Coffee Shop,Chinese Restaurant,Department Store,Bus Station,Accessories Store,Motel,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
9,Scarborough,0.0,College Stadium,Skating Rink,General Entertainment,Café,Monument / Landmark,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
15,Scarborough,0.0,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Breakfast Spot,Electronics Store,Bank,Sandwich Place,Pizza Place,Coffee Shop,Noodle House
18,North York,0.0,Coffee Shop,Clothing Store,Juice Bar,Restaurant,Bank,Theater,Shopping Mall,Food Court,Electronics Store,Liquor Store
19,North York,0.0,Café,Chinese Restaurant,Bank,Japanese Restaurant,Accessories Store,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
22,North York,0.0,Ramen Restaurant,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Fast Food Restaurant,Juice Bar,Bank,Restaurant,Plaza
24,North York,0.0,Pizza Place,Coffee Shop,Discount Store,Pharmacy,Grocery Store,Park,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant
26,North York,0.0,Gym,Café,Caribbean Restaurant,Dessert Shop,Japanese Restaurant,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
27,North York,0.0,Gym,Coffee Shop,Restaurant,Chinese Restaurant,Discount Store,Beer Store,Supermarket,Bike Shop,Clothing Store,Italian Restaurant


In [95]:
tor_data.loc[tor_data['Cluster Labels'] == 1, tor_data.columns[[1] + list(range(5, tor_data.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Playground,Park,Intersection,Accessories Store,Monument / Landmark,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
20,North York,1.0,Park,Accessories Store,Monument / Landmark,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
23,North York,1.0,Park,Convenience Store,Accessories Store,Modern European Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
40,East York/East Toronto,1.0,Park,Convenience Store,Intersection,Accessories Store,Monument / Landmark,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
50,Downtown Toronto,1.0,Park,Playground,Trail,Accessories Store,Modern European Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant
74,York,1.0,Park,Women's Store,Pool,Accessories Store,Modern European Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant


In [96]:
tor_data.loc[tor_data['Cluster Labels'] == 2, tor_data.columns[[1] + list(range(5, tor_data.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Print Shop,Fast Food Restaurant,Accessories Store,Modern European Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
1,Scarborough,2.0,Bar,Home Service,Accessories Store,Monument / Landmark,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
2,Scarborough,2.0,Medical Center,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Mexican Restaurant,Restaurant,Donut Shop,Bank,Mobile Phone Shop
4,Scarborough,2.0,Caribbean Restaurant,Bakery,Bank,Gas Station,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Lounge,Hakka Restaurant,Men's Store
5,Scarborough,2.0,Playground,Cosmetics Shop,Accessories Store,Monument / Landmark,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
7,Scarborough,2.0,Bakery,Bus Station,Intersection,Park,Metro Station,Ice Cream Shop,Bus Line,Accessories Store,Mobile Phone Shop,Miscellaneous Shop
8,Scarborough,2.0,Motel,American Restaurant,Monument / Landmark,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
10,Scarborough,2.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Office,Organic Grocery,Martial Arts School,Massage Studio,Medical Center
11,Scarborough,2.0,Auto Garage,Smoke Shop,Shopping Mall,Middle Eastern Restaurant,Bakery,Sandwich Place,Modern European Restaurant,Massage Studio,Medical Center,Mediterranean Restaurant
12,Scarborough,2.0,Lounge,Breakfast Spot,Latin American Restaurant,Clothing Store,Skating Rink,Accessories Store,Monument / Landmark,Medical Center,Mediterranean Restaurant,Men's Store


In [97]:
tor_data.loc[tor_data['Cluster Labels'] == 4, tor_data.columns[[1] + list(range(5, tor_data.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,4.0,Baseball Field,Food Truck,Accessories Store,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
91,Etobicoke,4.0,Baseball Field,Accessories Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
97,North York,4.0,Baseball Field,Food Service,Accessories Store,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
